# Imports

In [1]:
import prophet
import sys
sys.modules['fbprophet'] = prophet

In [2]:
import numpy as np
import pandas as pd

from collections import OrderedDict
import itertools
from datetime import datetime

import sys
sys.path.append("..")

from copy import deepcopy

from configs.kpi_constraints_dict import kpi_constraints_dict
from configs.bad_direction_kpi_dict import bad_direction_kpi_dict
from configs.functions import make_future, run_prophet_funct
from configs.functions import add_cond, mult_cond, is_weekday
from configs.functions import preprocess_data, hparam_tuning


import logging, sys, os
import time

from uuid import uuid4

import dask
from dask.distributed import Client
from sklearn.preprocessing import StandardScaler
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Dask client init

In [3]:
client = Client( dashboard_address = ':44594', n_workers = 68, threads_per_worker = 2 ) #scheduler_address=':37243'
client

/Users/eaxxprx/miniconda3/envs/nprophet/lib/python3.6/site-packages/distributed/node.py:155: UserWarning:

Port 44594 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53831 instead



Client Scheduler: tcp://127.0.0.1:53832 Dashboard: http://127.0.0.1:53831/status,Cluster Workers: 68 Cores: 136 Memory: 17.18 GB


# Data

In [25]:
metadata_store = pd.read_csv('/Users/eaxxprx/Desktop/Work/EEA Code/aio/Data/metadata_anon.csv')
metadata_store = metadata_store[ metadata_store.model_type == 'seasonal_prophet' ]
#metadata_store.dimension_name = metadata_store.dimension_name.map(lambda element: eval(element))

In [26]:
metadata_store.dimension_name.unique()

array([nan], dtype=object)

In [27]:
file = "4weeks-lte_enodeb_id-anon.csv"
datas = pd.read_csv( os.path.join("..", "data", file) )
print("-"*30,"DF READ ✔️","-"*30)

------------------------------ DF READ ✔️ ------------------------------


# Choose mdrows

In [34]:
mdrow_list = []

# arr of unique kpis present in file
#kpis = metadata_store.loc[ metadata_store['path'] == file ]['kpi_name'].unique()
kpis = metadata_store['kpi_name'].unique()

timeseries_counter = 0
delayed_result_list = []
for kpi in kpis: #test on 5:6
    #kpi = "tcp_tp_ul_sum"
    # mask out part of metadata df
    mask =  ((metadata_store['kpi_name'] == kpi) &       
            (metadata_store['model_type'] == 'seasonal_prophet') )#Paramesh: 2nd file check

    metadata_store_kpi = metadata_store.loc[ mask ]


    # in case of mutiple simensions, we need another for here for the keys of the dimension dict e.g.

    metadata_store_kpi['dim_str'] = metadata_store_kpi.apply(lambda x: '_'.join(
        [str(elem) for elem in x['dimension_name'].keys()]), axis=1)  #Paramesh: removed keys

    dim_names_arr = metadata_store_kpi['dim_str'].unique()


    for dimension_name in dim_names_arr:
        metadata_store_dim = metadata_store_kpi.loc[ metadata_store_kpi['dim_str'] == dimension_name ]

        
        for _, row in metadata_store_dim.iterrows():
            timeseries_counter += 1
            rowd = row.to_dict()
            
            processing_status = {"ts_uuid": str(uuid4()), "failed": False, "num_fails":0, "last_failed_ts":None}

            row = {**rowd, **processing_status}
            #row = pd.concat([x, row])
            mdrow_list.append(row)

AttributeError: 'float' object has no attribute 'keys'

In [36]:
metadata_store_kpi['dimension_name'].unique()

array([nan], dtype=object)

In [29]:
metadata_store_kpi.columns

Index(['Unnamed: 0', 'ts', 'path', 'dimension_name', 'kpi_name',
       'missing_data_ratio_all', 'missing_data_ratio_last_week',
       'seasonality_flag', 'statonarity_flag', 'missing_data_imputation_flag',
       'table', 'nan_trimming_flag', 'ACF_max_difference', 'is_it_constant',
       'model_type'],
      dtype='object')

# Params

In [9]:
missing_data_percentage_param = 0.3
DAILY_FOURIER_ORDER = 3
WEEKLY_FOURIER_ORDER = 5
COUNTRY_NAME = 'USA'
percent = 0.1
scores = ['mae'] #['mdape', 'mape', 'smape', 'mae']
predictions_write_to = '' 
errors_write_to = ''
write_to = 'prophet_results2/'

In [10]:
end = pd.to_datetime(metadata_store['ts'].values[0], unit='s')
ts = metadata_store['ts'].values[0]
start = end - pd.Timedelta(4, unit = 'w')

In [11]:
param_grid =    {  
                'changepoint_prior_scale': [0.01, 0.1, 1.0],
                'seasonality_prior_scale': [0.1, 1.0, 10.0, 50],
                'seasonality_mode' : ['additive', 'multiplicative'],
                }

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
all_params2 = [dict(zip(list(param_grid.keys())+['weekend'], v)) for v in itertools.product(*list(param_grid.values())+[['True', 'False']])]

In [12]:
p_uuid_df = pd.DataFrame( { str(uuid4()): p  for p in all_params2}).T   # p_uuid: 1 id per parameter combination  for tracking - same for all models with the same hparams, regardless of timeseries

p_uuid_df.iloc[0].to_dict()
for idx, row in p_uuid_df.head().iterrows():
    print(idx, row.to_dict())

ea2ac5b3-668f-44e7-86df-b88351b15268 {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.1, 'seasonality_mode': 'additive', 'weekend': 'True'}
04c43ce7-9e6b-4596-baa3-95cbca90bebc {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.1, 'seasonality_mode': 'additive', 'weekend': 'False'}
87397db6-d60c-4875-a613-c82cc0c835f0 {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.1, 'seasonality_mode': 'multiplicative', 'weekend': 'True'}
8e6b642b-a369-4273-8b34-3a1f5d8d3ad8 {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.1, 'seasonality_mode': 'multiplicative', 'weekend': 'False'}
f34b4842-917c-4fa2-a89a-096884e2a5a8 {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 1.0, 'seasonality_mode': 'additive', 'weekend': 'True'}


In [13]:
p_uuid_df.head()

,changepoint_prior_scale,seasonality_prior_scale,seasonality_mode,weekend
ea2ac5b3-668f-44e7-86df-b88351b15268,0.01,0.1,additive,True
04c43ce7-9e6b-4596-baa3-95cbca90bebc,0.01,0.1,additive,False
87397db6-d60c-4875-a613-c82cc0c835f0,0.01,0.1,multiplicative,True
8e6b642b-a369-4273-8b34-3a1f5d8d3ad8,0.01,0.1,multiplicative,False
f34b4842-917c-4fa2-a89a-096884e2a5a8,0.01,1,additive,True


# Functions

In [14]:
def submit_training(df, row, p_uuid_df):

    df_p_dict = {} # store and return performance metrics of models
    
    # submit tasks to the client
    for p_uuid, params_row in p_uuid_df.iterrows(): # we use all_params2 here, which have the weekend param too
        #try:
        params = params_row.to_dict()
        df_p = hparam_tuning(df, params, row,
                                is_weekend = True,
                                parallel = "dask",
                                daily_fourier_order=DAILY_FOURIER_ORDER,
                                weekly_fourier_order=WEEKLY_FOURIER_ORDER)
        #_params = { **params, **{"weekend": True}}
        df_p_dict[ p_uuid ] = df_p

        
    return df_p_dict

In [15]:
mdrow_list

[]

In [16]:
pd.DataFrame(mdrow_list).head(5)

""


In [17]:
mdrow_df = pd.DataFrame(mdrow_list)[:10]
#mdrow_df.head()

In [18]:
_df_dict = {}
for  _, mdrow in mdrow_df[:500].iterrows():
    df = dask.delayed( preprocess_data)( datas, mdrow, start, end)
    _df_dict[ mdrow.ts_uuid ] = df
df_dict = dask.compute( _df_dict )[0]
df_dict.keys()

dict_keys([])

In [19]:
#q = submit_training(df_dict["66b8eea5-ffe0-4918-82de-5a2b7aff1336"], mdrow_df.loc[0], p_uuid_df)

In [20]:
#len(compute['66b8eea5-ffe0-4918-82de-5a2b7aff1336'])

# Compute

In [21]:
taskgroups = {}

for ts_uuid in mdrow_df.ts_uuid:
    df = df_dict[ ts_uuid ]
    mdrow = mdrow_df[ mdrow_df.ts_uuid == ts_uuid ]

    tg = dask.delayed(submit_training)(df, mdrow, p_uuid_df)
    taskgroups[ ts_uuid ] = tg

compute = dask.compute(taskgroups)[0]

# 500 -> 3400 s
# 20  -> 140 s, 210 s

AttributeError: 'DataFrame' object has no attribute 'ts_uuid'

### Note:
Unfortunately sometimes tasks are stuck pending. This happens mostly on smaller workloads, perhaps the timeout is shorter in these cases.

In [ ]:
[ [ fut.status != "finished" for p_uuid, fut in compute[ts_uuid].items()] for ts_uuid in compute.keys() ]       # task status
[ all([ fut.status == "finished" for p_uuid, fut in compute[ts_uuid].items()]) for ts_uuid in compute.keys() ]  # taskgroup status

In [ ]:
q = client.gather(compute)

In [ ]:
#q

In [ ]:
pd.Series([ [ fut.status for p_uuid, fut in compute[ts_uuid].items()] for ts_uuid in compute.keys() ]).explode().value_counts()

In [ ]:
zookeeper_table = pd.DataFrame(columns = ["ts_uuid", "p_uuid", "ts", "kpi", "dim", "params", "failed", "num_failed", "last_failed_ts"]) 
# here ts in ts_uuid stands for timeseries, while ts is timestamp. 🦆 me
# failed contains the status of the last retry

In [ ]:
[[ p_uuid for p_uuid, fut in compute[ts_uuid].items() if fut.status != "finished"] for ts_uuid in compute.keys() ]

In [ ]:
zookeeper_table

In [ ]:
def tg_finished(ts_uuid):
    """Checking if all tasks are finish"""
    return all([ fut.status == "finished" for p_uuid, fut in compute[ts_uuid].items()])

In [ ]:
res = {}

# TODO invoke manual retry for failed tasks
for ts_uuid, tg in compute.items():

    # if all models (tasks) finished for a timeseries (taskgroup), 
    # and the result was not acquired from distributed before
    if tg_finished(ts_uuid) and (ts_uuid not in res):

        try: #!
            r = { p_uuid: task.result(timeout = 3) for p_uuid, task in tg.items() }     # gather result from distributed by our task keys
            for p_uuid in r:
                params = p_uuid_df.loc[p_uuid].to_dict()        # ask for params by uuid key
                r[ p_uuid ]["params"] = pd.Series({0: params})      # 💩
            res[ts_uuid] = pd.concat( r ).reset_index(drop = True)
        except Exception as e:
            print("Valami 🐕 nagy hiba van: ", e)

    # there is at least 1 failed model (task)
    else:


        for p_uuid, task in tg.items():


            if task.status == "finished":
                # the task is no longer failed
                if (ts_uuid in zookeeper_table.ts_uuid.values) and (p_uuid in zookeeper_table.p_uuid.values): # task was previously in zookeper_table
                    mask = ((ts_uuid == zookeeper_table.ts_uuid) & (p_uuid == zookeeper_table.p_uuid))
                    zk_row = zookeeper_table[ mask ]
                    zk_row.failed = False
                    zookeeper_table[ mask ] = zk_row
                    # TODO put back finished future into "compute" dict
                
                # the task is finished, but some other tasks in the taskgroup failed: do nothing with task
                else:
                    pass
                                                    
            else:
                # task has failed and had failed before
                if (ts_uuid in zookeeper_table.ts_uuid.values) and (p_uuid in zookeeper_table.p_uuid.values):
                    mask = ((ts_uuid == zookeeper_table.ts_uuid) & (p_uuid == zookeeper_table.p_uuid)) # task was previously in zookeper_table
                    zk_row = zookeeper_table[ mask ]
                    zk_row.num_failed += 1
                    zk_row.last_failed_ts = int(time.time())
                    zookeeper_table[ mask ] = zk_row
                    # TODO do not increase num failed above a certain threshold (say 10), halt manual retries for the task

                # new task, first fail
                else:
                    mdrow = mdrow_df[ mdrow_df.ts_uuid == ts_uuid ]
                    task_to_retry = { "ts_uuid": ts_uuid, "p_uuid": p_uuid, "ts": mdrow.ts.values[0], 
                                    "kpi": mdrow.kpi_name.values[0], "dim": mdrow.dimension_name.values[0],
                                    "params": p_uuid_df.loc[p_uuid].to_dict(), "failed": True,
                                    "num_failed": 1, "last_failed_ts": int(time.time())}
                    zookeeper_table = zookeeper_table.append(pd.Series(task_to_retry), ignore_index=True)



In [ ]:
to_retry = {}
for _, zk_row in zookeeper_table.iterrows():
    df = df_dict[ zk_row.ts_uuid ]
    params = p_uuid_df.loc[ zk_row.p_uuid ].to_dict()
    row = mdrow_df[ mdrow_df.ts_uuid == zk_row.ts_uuid ]

    if zk_row.ts_uuid not in to_retry:
        to_retry[ zk_row.ts_uuid ] = {}

    df_p = dask.delayed(hparam_tuning)(df, params, row,
                                is_weekend = True,
                                parallel = "dask",
                                daily_fourier_order=DAILY_FOURIER_ORDER,
                                weekly_fourier_order=WEEKLY_FOURIER_ORDER)
    to_retry[ zk_row.ts_uuid ][ zk_row.p_uuid ] = df_p
compute_retry = dask.compute(to_retry)[0]

In [ ]:
compute_retry

In [ ]:
r[ p_uuid ]["params"] = pd.Series({0: params})
r[ p_uuid ]

In [ ]:

#res['6f371025-8dc3-408c-a0a2-ac88c07e02fd']

In [ ]:
res

In [ ]:
#pd.Series([ { k[kk].status for kk in compute[k]} for k in compute.keys() ]).explode().value_counts()
#rrrr["params"] = all_params[0]
rrrr.params = pd.Series({0:all_params2[0]})
rrrr

In [ ]:
zookeeper_table

In [ ]:
mdrow_df[ mdrow_df.ts_uuid == "443685ea-4430-4a05-a34d-1af6360c37f1" ]

# Az elvarratlan thread-ek

In [ ]:
mdrow_df["failed"] = mdrow_df.uuid.map( lambda uuid: is_taskgroup_failed( compute[uuid] ) )
mdrow_df["num_fails"] = mdrow_df.num_fails.copy() + mdrow_df.failed
mdrow_df["last_failed_ts"] = mdrow_df.uuid.map( lambda uuid: datetime.now().strftime("%d/%m/%Y %H:%M:%S") 
                                                                if is_taskgroup_failed( compute[uuid] )
                                                                else mdrow_df[mdrow_df["uuid"]==uuid].last_failed_ts.values[0] )


In [ ]:
[ [ (uuid,all_params2[idx]) for idx in range(len(tg)) if tg[idx][0].status != "finished" ] for uuid, tg in compute.items()]

In [ ]:
[[ client.retry(task) for task in tg if task.status!= "finished" ] for tg in compute ]

In [ ]:
mdrow_df["failed"] = [ is_taskgroup_failed(tg) for tg in compute ]
#mdrow_df[""]

In [ ]:
from dask.distributed import TimeoutError, CancelledError


queue_idx = {*zip(range(20), [str(uuid4()) for _ in range(20)])}
results = np.empty(shape = 20, dtype = pd.DataFrame)

#compute 

while queue_idx: #legyen valami kilépés ne while 
    task_list = []
    for idx in queue_idx:
        task = dask.delayed(submit_training)(df_list[ idx ], mdrow_list[ idx ], all_params)
        task_list.append(task)
    
    queue = dask.compute(task_list)[0]
    
    time.sleep(30)
    for idx in queue_idx:
        
        results_list = []
        taskgroup = queue[ idx ]

        ok = True                   # If any task in taskgroup fails -> continue loop: idx++
        for task in taskgroup:
            try:
                result = task.result(timeout = 3)
                results_list.append(result)
                
            except (TimeoutError, CancelledError) as e:
                print(idx," ",e, sep = "")
                ok = False
                break
                
        if not ok:
            continue

        queue_idx.remove( idx )     #remove idx from queue if success

        tuning_results = pd.concat( results_list ).reset_index(drop=True)
        results[ idx ] = tuning_results

In [ ]:
results_concat = [ pd.concat(elem).reset_index(drop = True) for elem in compute ]

In [ ]:
df_list[0]

In [ ]:
tuning_results = results_concat[0].copy()
tuning_results["params"] = deepcopy(all_params2)
score = "mae"

myForecast(df_list[0], tuning_results, mdrow_list[0], "mae")
#tuning_results.params.map(lambda x: x["weekend"] if "weekend" in x else 0)

In [ ]:
def myForecast(df, tuning_results,
               row, score):

    kpi = row["kpi_name"]
    dim_dict = row["dimension_name"]

    multiplicative_condition = False
    additive_condition = False
    lower = kpi_constraints_dict[kpi][0]
    upper = kpi_constraints_dict[kpi][1]

    tuning_results[score+'_rank'] = tuning_results[score].rank()
    tuning_results['rank'] = tuning_results[score+'_rank']

    params = tuning_results.loc[ tuning_results["rank"].idxmin(), "params" ]
    weeknd = params['weekend']
    params.pop("weekend", None)

    if weeknd:
        m = run_prophet_funct(df, params, 
                                DAILY_FOURIER_ORDER, 
                                WEEKLY_FOURIER_ORDER,
                                True,
                                COUNTRY_NAME)
        future = make_future(m, end, 168)
        future['weekday'] = future['ds'].apply(is_weekday)
        future['weekend'] = ~future['ds'].apply(is_weekday)
        forecast = m.predict(future)
    
    else:
        m = run_prophet_funct(df, params, 
                                DAILY_FOURIER_ORDER, 
                                WEEKLY_FOURIER_ORDER, 
                                False, 
                                COUNTRY_NAME)
        future = make_future(m, end, 168)
        forecast = m.predict(future)
    
    if params['seasonality_mode'] == 'multiplicative':
        multiplicative_condition = mult_cond(forecast, df, percent, lower, upper, end)# ha mult_cond - > True, akkor NEM multiplikatív
        #ellenorizzuk a feltetelek teljesuleset
        if multiplicative_condition:
            
            tuning_results_additive = tuning_results.loc[tuning_results['seasonality_mode']=='additive']
            #majd kivalasztjuk a legjobbat
            params = tuning_results_additive[list(param_grid.keys())+['weekend']].loc[tuning_results_additive['rank'].idxmin()]
            #es tanitunk, vot ma
            weeknd = params['weekend']
            if weeknd:
                m = run_prophet_funct(df, params[:-1], 
                                        DAILY_FOURIER_ORDER, 
                                        WEEKLY_FOURIER_ORDER,
                                        is_weekend = True, 
                                        country_name = COUNTRY_NAME)
                future = make_future(m, end, 168)
                future['weekday'] = future['ds'].apply(is_weekday)
                future['weekend'] = ~future['ds'].apply(is_weekday)
                forecast = m.predict(future)

            else:
                m = run_prophet_funct(df, params[:-1], 
                                        DAILY_FOURIER_ORDER, 
                                        WEEKLY_FOURIER_ORDER, 
                                        is_weekend = False, 
                                        country_name = COUNTRY_NAME)
                future = make_future(m, end, 168)
                forecast = m.predict(future)
    
    if params['seasonality_mode'] == 'additive':
        additive_condition = add_cond(forecast, percent, lower, upper)
        #check conditions and fix
        if additive_condition:
            if len(m.changepoints[np.abs(np.nanmean(m.params['delta'], axis=0)) >= 0.01].values)==0:
                last_changepoint = start
            else:
                last_changepoint = m.changepoints[np.abs(np.nanmean(m.params['delta'], axis=0)) >= 0.01].values[-1]
            minimum = forecast.set_index('ds')[end:]['additive_terms'].quantile(0.05)
            maximum = forecast.set_index('ds')[end:]['additive_terms'].quantile(0.95)
            last_point =((forecast.set_index('ds')[last_changepoint:]['trend']+minimum>lower)
                    & (forecast.set_index('ds')[last_changepoint:]['trend']+maximum<upper))[::-1].idxmax()

            forecast.loc[forecast['ds']>last_point, 'trend'] = forecast.loc[forecast['ds']==last_point, 'trend'].values[0]
            forecast['yhat'] = (forecast['trend']*(1+forecast['multiplicative_terms'])+forecast['additive_terms'])
    
    forecast['yhat'] = forecast['yhat'].clip(lower = lower, upper = upper)
    
    
    scaler =  StandardScaler(with_mean = False) # RobustScaler
    scaler.fit(df['y'].values.reshape(-1,1))

    df = df.set_index("ds")
    forecast = forecast.set_index("ds")

    results = pd.DataFrame( index = forecast.index,
                            columns = ["kpi_name", "dimension_name", "ground_truth",
                                        "pred", "error", "trend", "gt_wo_trend", "pred_wo_trend"])

    results["kpi_name"] = [kpi] * len(results)
    results["dimension_name"] = [dim_dict] * len(results)
    results["y_raw"] = df.y

    #kpi név, dim , ts, trend, szezon, zaj, holiday, eredeti y
    return forecast

    #pd.Series(scaler.transform((df.set_index('ds')['y']-forecast.set_index('ds')['yhat']).values.reshape(-1,1)).T[0], index=forecast['ds'], name = dims_values_str))
    #scaler.transform((df['y']-forecast['yhat']).values.reshape(-1,1)).T[0]


In [ ]:
from distributed.client import Future
#isinstance(results[0][0], Future)
[ client.gather(elem) for elem in compute ]

In [ ]:
import time
from uuid import uuid4

In [ ]:

queue_idx

In [ ]:


"""while failed:
    for idx in failed:
        task_list = dask.delayed(submit_training)(df_list[ idx ], mdrow_list[ idx ], all_params)
    retry_queue = dask.compute(task_list)[0]"""

In [ ]:


def gather_or_fail(queue, retried = False):
    """
    Input:
        queue - list of np.ndarray of dask.distributed.client.Future of unspecified status.
        Each element of the np.ndarry is a one-rowed performance_metrics pd.DataFrame.
    
    Output:
        results - np.ndarray of pd.DataFrames.
        failed_tasks - list of np.ndarray of failed dask futures with TimeoutError and CancelledError.
    """
    results = np.empty(shape = len(queue), dtype = pd.DataFrame)
    failed_tasks = []

    cycle_vars = enumerate(queue)

    if retried:
        cycle_vars = 

    for idx, task in cycle_vars:
        
        results_list = []
        for subtask in task:
            
            try:
                result = subtask.result(timeout = 3)
                
            except (TimeoutError, CancelledError) as e:
                failed_tasks.append(idx)
                print(idx," ",e, sep = "")
                break
                
            results_list.append(result)
        
        tuning_results = pd.concat( results_list ).reset_index(drop=True)
        results[ idx ] = tuning_results

In [ ]:
enumerate(q)

In [ ]:
while failed_tasks:
    for idx in failed_tasks:
        qq = dask.delayed(submit_training)(df_list[ idx ], mdrow_list[ idx ], all_params)
    results, failed_tasks = gather_or_fail(qq)

In [ ]:
failed_tasks

In [ ]:
results = [ [  x.result() for x in subarr ] for subarr in q ]

In [ ]:
qq = dask.delayed(submit_training)(df_list[ 16 ], mdrow_list[ 16 ], all_params).compute()

In [ ]:
for idx,subarr in enumerate(q):
    print([ x.result() for x in subarr])
    print(idx)

In [ ]:
results = [ pd.concat(x).reset_index(drop=True) for x in results ]

In [ ]:
top3s = []
for tuning_results in results:
    tuning_results["params"] = all_params2
    tuning_results["rank"] = tuning_results.mae.rank()
    top3s.append( tuning_results.sort_values("rank").head(1) )

In [ ]:
pd.concat(top3s).reset_index(drop=True).params.map( lambda d: d["seasonality_prior_scale"]).value_counts()

In [ ]:

q = hparam_tuning(df_list[0], all_params[0], mdrow_list[0],
                                    parallel = "dask",
                                    daily_fourier_order=DAILY_FOURIER_ORDER,
                                    weekly_fourier_order=weekly_fourier_order)

q.result()

# Todo

TS.model <-> paraméter összekapcsolás

# Original

In [ ]:
#infos
end = pd.to_datetime(metadata_store['ts'].values[0], unit='s')
ts = metadata_store['ts'].values[0]
start = end - pd.Timedelta(4, unit = 'w')
files = set(metadata_store['path'])
for file in files:
    datas = pd.read_csv(read_from+file)
    kpis = set(metadata_store.loc[metadata_store['path']==file]['kpi_name'])
    for kpi in kpis:
        all_errors =pd.DataFrame(index = pd.date_range(start, end, freq = 'H'))
        all_predictions = pd.DataFrame(index = pd.date_range(start, end+pd.Timedelta('1w'), freq = 'H'))
        standard_deviations = {}
        metadata_store_kpi = metadata_store.loc[(metadata_store['kpi_name']==kpi) & (metadata_store['path']==file) &(metadata_store['model_type']=='seasonal_prophet')]
        # in case of mutiple simensions, we need another for here for the keys of the dimension dict e.g.
        #metadata_store_kpi['dim_str'] = metadata_store_kpi.apply(lambda x: '_'.join([str(elem) for elem in eval(x['dimension_name']).keys()]))
        #dim_names =  set(metadata_store_kpi['dim_str'])
        #for dimension_name in dim_names: 
        for index, row in metadata_store_kpi.loc[metadata_store['model_type']=='seasonal_prophet'].iterrows():

                dim_dict = eval(row['dimension_name'])
                dims_str = '_'.join([str(elem) for elem in dim_dict.keys()])
                dims_values_str = '_'.join([str(elem) for elem in dim_dict.values()])

                multiplicative_condition = False
                additive_condition = False
                lower = kpi_constraints_dict[kpi][0]
                upper = kpi_constraints_dict[kpi][1]
                
                df = pd.DataFrame()

                data = datas.loc[(datas[list(dim_dict)] == pd.Series(dim_dict)).all(axis=1)]
                df['y'] = data.dropna(axis=0, subset =[kpi])[kpi]

                df['ds'] = pd.to_datetime(data.dropna(axis=0, subset =[kpi])["ts"], unit='s')

                df = df.loc[(df['ds']>=start)&(df['ds']<=end)]
                df = df.sort_values('ds')
                df = df.reset_index()
                scaler =  StandardScaler(with_mean = False)
                scaler.fit(df['y'].values.reshape(-1,1)) 
                standard_deviations[dims_str] = scaler.scale_



                param_grid = {  
                'changepoint_prior_scale': [0.01, 0.1, 1.0],
                'seasonality_prior_scale': [0.1, 1.0, 10.0, 50],
                'seasonality_mode' : ['additive', 'multiplicative']
                }
                all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
                scores_dict = {i:[] for i in scores} #{'mape':[0.012, 0.07,...], 'smpae': []}
                for params in all_params:
                    m = run_prophet_funct(df, params, daily_fourier_order, weekly_fourier_order, True,  country_name)
                    if float(row['missing_data_ratio_all'])<0.15:
                        df_cv = cross_validation(m,initial='16 days', horizon='2 days', period = '2 days', parallel="processes")
                    else:
                        df_cv = cross_validation(m,initial='16 days', horizon='1 days', period = '1 days', parallel="processes")
                    df_p = performance_metrics(df_cv, rolling_window=1)




                    for i in scores:
                        if i!='mape':
                            scores_dict[i].append(df_p[i].values[0])
                        elif (i == 'mape') and ('mape' in df_p.columns):
                            scores_dict[i].append(df_p[i].values[0])
                        else:
                            scores = [i for i in scores if i!='mape']



                    m = run_prophet_funct(df, params, daily_fourier_order, weekly_fourier_order, False, country_name)
                    if float(row['missing_data_ratio_all'])<0.15:
                        df_cv = cross_validation(m,initial='16 days', horizon='2 days', period = '2 days', parallel="processes")
                    else:
                        df_cv = cross_validation(m,initial='16 days', horizon='1 days', period = '1 days', parallel="processes")
                    df_p = performance_metrics(df_cv, rolling_window=1)
                    for i in scores:
                        if i!='mape':
                            scores_dict[i].append(df_p[i].values[0])
                        elif (i == 'mape') and ('mape' in df_p.columns):
                            scores_dict[i].append(df_p[i].values[0])
                        else:
                            scores = [i for i in scores if i!='mape']

                #megnezzuk, hogy melyik parameter a legjobb, hozzaveve a weekend-weekday elvalasztast is
                all_params = [dict(zip(list(param_grid.keys())+['weekend'], v)) for v in itertools.product(*list(param_grid.values())+[['True', 'False']])]
                tuning_results = pd.DataFrame(all_params)
                #vegigmegyunk a scoreokon:  
                for score in scores:
                    tuning_results[score] = scores_dict[score]
                    tuning_results[score+'_rank'] = tuning_results[score].rank()
                    tuning_results['rank'] = tuning_results[score+'_rank']
                    #azt valasztjuk, amit a legelorebb sorolt, ha tobb van veletlenul valasztunk
                    params = tuning_results[list(param_grid.keys())+['weekend']].loc[tuning_results['rank'].idxmin()]
                    #mivel ezt kulon kell futattni, elmentjuk, hogy van-e weekend
                    weeknd = params['weekend']
                    #ujra fut a prophet
                    if weeknd:
                        m = run_prophet_funct(df, params[:-1], daily_fourier_order, weekly_fourier_order, True, country_name)
                        future = make_future(m, end, 168)
                        future['weekday'] = future['ds'].apply(is_weekday)
                        future['weekend'] = ~future['ds'].apply(is_weekday)
                        forecast = m.predict(future)
                        

                    else:
                        m = run_prophet_funct(df, params[:-1], daily_fourier_order, weekly_fourier_order, False, country_name)
                        future = make_future(m, end, 168)
                        forecast = m.predict(future)
                    
                    
                    if params['seasonality_mode'] == 'multiplicative':
                        multiplicative_condition = mult_cond(forecast, df, percent, lower, upper)
                        #ellenorizzuk a feltetelek teljesuleset
                        if multiplicative_condition:
                            #? Miért a seasonality_mode == "additive" elemeit nézzük itt?

                            #! Mintha meg lenne ismételve a forecastolás itt és egy tabbal kijjebb
                            tuning_results_additive = tuning_results.loc[tuning_results['seasonality_mode']=='additive']
                            #majd kivalasztjuk a legjobbat
                            params = tuning_results_additive[list(param_grid.keys())+['weekend']].loc[tuning_results_additive['rank'].idxmin()]
                            #es tanitunk, vot ma
                            weeknd = params['weekend']
                            if weeknd:
                                m = run_prophet_funct(df, params[:-1], daily_fourier_order, weekly_fourier_order,is_weekend =  True, country_name = country_name)
                                future = make_future(m, end, 168)
                                future['weekday'] = future['ds'].apply(is_weekday)
                                future['weekend'] = ~future['ds'].apply(is_weekday)
                                forecast = m.predict(future)

                            else:
                                m = run_prophet_funct(df, params[:-1], daily_fourier_order, weekly_fourier_order, False, country_name)
                                future = make_future(m, end, 168)
                                forecast = m.predict(future)
                            

                    if params['seasonality_mode'] == 'additive':
                        additive_condition = add_cond(forecast, percent, lower, upper)
                        #check conditions and fix
                        if additive_condition:
                            if len(m.changepoints[np.abs(np.nanmean(m.params['delta'], axis=0)) >= 0.01].values)==0:
                                last_changepoint = start
                            else:
                                last_changepoint = m.changepoints[np.abs(np.nanmean(m.params['delta'], axis=0)) >= 0.01].values[-1]
                            minimum = forecast.set_index('ds')[end:]['additive_terms'].quantile(0.05)
                            maximum = forecast.set_index('ds')[end:]['additive_terms'].quantile(0.95)
                            last_point =((forecast.set_index('ds')[last_changepoint:]['trend']+minimum>lower)
                                 & (forecast.set_index('ds')[last_changepoint:]['trend']+maximum<upper))[::-1].idxmax()

                            forecast.loc[forecast['ds']>last_point, 'trend'] = forecast.loc[forecast['ds']==last_point, 'trend'].values[0]
                            forecast['yhat'] = (forecast['trend']*(1+forecast['multiplicative_terms'])+forecast['additive_terms'])
                    forecast['yhat'] = forecast['yhat'].clip(lower = lower, upper = upper)
                    all_predictions = all_predictions.join(forecast[['ds', 'yhat']].rename(columns={'yhat':dims_values_str}).set_index('ds'))
                    all_errors = all_errors.join(pd.Series(scaler.transform((df.set_index('ds')['y']-forecast.set_index('ds')['yhat']).values.reshape(-1,1)).T[0], index=forecast['ds'], name = dims_values_str))
                    





    